<h1 style='background:#1EA1A1; border:0; color:white'><center> 📝 Abstract <center></h1>

 * In this Training Notebook EfficientNetB1 is Trained.
 *[Inference Notebook](https://www.kaggle.com/bharadwajvedula/lb-0-8716-tpu-tf-efficientnetb1-inference)

**Versions**    
* Version 2: EfficientNetB1 512 Augs CV:- **0.8684**  LB:- **0.8716**





**References are mentioned at the end.**


<a id="0"></a>
<h1 style='background:#1EA1A1; border:0; color:white'><center> 📄 Table of Contents<center></h1>

* [Imports](#1)
* [Configs](#2)
* [Reading TFRecords](#3) 
* [Dataset Creation](#4)
* [Modelling](#5)
* [KFold Prediction](#6)
* [References](#7)

<a id="1"></a>
<h1 style='background:#1EA1A1; border:0; color:white'><center> 🚚 Imports<center></h1>

In [ ]:
!pip install efficientnet tensorflow_addons > /dev/null

In [ ]:
import os 
import re
import warnings
from glob import glob
from tqdm.auto import tqdm 
from scipy.signal import get_window
from typing import Optional, Tuple
from kaggle_datasets import KaggleDatasets


import numpy as np 
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split,StratifiedKFold,KFold

import tensorflow as tf
import tensorflow.keras.backend as K
from tensorflow.keras.models import Model , load_model, save_model 
from tensorflow.keras.layers import Input,Dropout,Conv2D,Flatten,GlobalAveragePooling2D,Dense,GlobalAvgPool2D
import efficientnet.tfkeras as efn
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, EarlyStopping
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import BinaryCrossentropy
import tensorflow_addons as tfa


<h1 style='background:#1EA1A1; border:0; color:white'><center> TPU Detection <center></h1>

In [ ]:
# Detect hardware, return appropriate distribution strategy
try:
    # TPU detection. No parameters necessary if TPU_NAME environment variable is
    # set: this is always the case on Kaggle.
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
    print('Running on TPU ', tpu.master())
except ValueError:
    tpu = None

if tpu:
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.experimental.TPUStrategy(tpu)
else:
    # Default distribution strategy in Tensorflow. Works on CPU and single GPU.
    strategy = tf.distribute.get_strategy()

print("REPLICAS: ", strategy.num_replicas_in_sync)

<a id="2"></a>
<h1 style='background:#1EA1A1; border:0; color:white'><center> ⚙️ Configs<center></h1>

In [ ]:
class config:
    seed = 2021
    replicas = strategy.num_replicas_in_sync
    batch_size = 32 * replicas
    AUTO = tf.data.experimental.AUTOTUNE
    dir_path = './result'
    epochs = 15
    #augmentations
    image_size = 512
    aug = True 
    mix_up_p = 0.1
    s_shift = 0.0
    t_shift = 0.0
    r_angle = 0 / 180 * np.pi
    label_positive_shift = 0.99


if not os.path.exists(config.dir_path):
    os.makedirs(config.dir_path)

def seed_everything(seed = config.seed):
    print(f"seeded everything to seed {seed}")
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    tf.random.set_seed(seed)

seed_everything()

<h1 style='background:#1EA1A1; border:0; color:white'><center> Extracting GCS Path <center></h1>

In [ ]:
# Dataset of Hidehisa Arai

train_files = ['gs://kds-fe725ba5a3259c712812aed413cfe61fc3827b135988e28e694e515c/train0.tfrecords',
 'gs://kds-fe725ba5a3259c712812aed413cfe61fc3827b135988e28e694e515c/train1.tfrecords',
 'gs://kds-fe725ba5a3259c712812aed413cfe61fc3827b135988e28e694e515c/train2.tfrecords',
 'gs://kds-fe725ba5a3259c712812aed413cfe61fc3827b135988e28e694e515c/train3.tfrecords',
 'gs://kds-fe725ba5a3259c712812aed413cfe61fc3827b135988e28e694e515c/train4.tfrecords',
 'gs://kds-ecd1c4515f350d79d5a8d2e9a07df52885278c6cce43ad116c8b03cb/train5.tfrecords',
 'gs://kds-ecd1c4515f350d79d5a8d2e9a07df52885278c6cce43ad116c8b03cb/train6.tfrecords',
 'gs://kds-ecd1c4515f350d79d5a8d2e9a07df52885278c6cce43ad116c8b03cb/train7.tfrecords',
 'gs://kds-ecd1c4515f350d79d5a8d2e9a07df52885278c6cce43ad116c8b03cb/train8.tfrecords',
 'gs://kds-ecd1c4515f350d79d5a8d2e9a07df52885278c6cce43ad116c8b03cb/train9.tfrecords',
 'gs://kds-2c0b3b314d8607851f6dddea1e976aab52547b68de5117cf3e76a85a/train10.tfrecords',
 'gs://kds-2c0b3b314d8607851f6dddea1e976aab52547b68de5117cf3e76a85a/train11.tfrecords',
 'gs://kds-2c0b3b314d8607851f6dddea1e976aab52547b68de5117cf3e76a85a/train12.tfrecords',
 'gs://kds-2c0b3b314d8607851f6dddea1e976aab52547b68de5117cf3e76a85a/train13.tfrecords',
 'gs://kds-2c0b3b314d8607851f6dddea1e976aab52547b68de5117cf3e76a85a/train14.tfrecords',
 'gs://kds-965e4184655c915b40b37bf7e5d3c8e310dd5751e8a2e1f5bdc8d70d/train15.tfrecords',
 'gs://kds-965e4184655c915b40b37bf7e5d3c8e310dd5751e8a2e1f5bdc8d70d/train16.tfrecords',
 'gs://kds-965e4184655c915b40b37bf7e5d3c8e310dd5751e8a2e1f5bdc8d70d/train17.tfrecords',
 'gs://kds-965e4184655c915b40b37bf7e5d3c8e310dd5751e8a2e1f5bdc8d70d/train18.tfrecords',
 'gs://kds-965e4184655c915b40b37bf7e5d3c8e310dd5751e8a2e1f5bdc8d70d/train19.tfrecords']

test_files = ['gs://kds-50bb183919e58675b28621ce8be69435cf20585a6d6cb1842bc006f5/test0.tfrecords',
 'gs://kds-50bb183919e58675b28621ce8be69435cf20585a6d6cb1842bc006f5/test1.tfrecords',
 'gs://kds-50bb183919e58675b28621ce8be69435cf20585a6d6cb1842bc006f5/test2.tfrecords',
 'gs://kds-50bb183919e58675b28621ce8be69435cf20585a6d6cb1842bc006f5/test3.tfrecords',
 'gs://kds-50bb183919e58675b28621ce8be69435cf20585a6d6cb1842bc006f5/test4.tfrecords',
 'gs://kds-becf38913abeb284aec05b255f7dc057aeb025941368bbcbc5f64bb0/test5.tfrecords',
 'gs://kds-becf38913abeb284aec05b255f7dc057aeb025941368bbcbc5f64bb0/test6.tfrecords',
 'gs://kds-becf38913abeb284aec05b255f7dc057aeb025941368bbcbc5f64bb0/test7.tfrecords',
 'gs://kds-becf38913abeb284aec05b255f7dc057aeb025941368bbcbc5f64bb0/test8.tfrecords',
 'gs://kds-becf38913abeb284aec05b255f7dc057aeb025941368bbcbc5f64bb0/test9.tfrecords']

In [ ]:
# Function to create cqt kernel
def create_cqt_kernels(
    q: float,
    fs: float,
    fmin: float,
    n_bins: int = 84,
    bins_per_octave: int = 12,
    norm: float = 1,
    window: str = "hann",
    fmax: Optional[float] = None,
    topbin_check: bool = True
) -> Tuple[np.ndarray, int, np.ndarray, float]:
    fft_len = 2 ** _nextpow2(np.ceil(q * fs / fmin))
    
    if (fmax is not None) and (n_bins is None):
        n_bins = np.ceil(bins_per_octave * np.log2(fmax / fmin))
        freqs = fmin * 2.0 ** (np.r_[0:n_bins] / np.float(bins_per_octave))
    elif (fmax is None) and (n_bins is not None):
        freqs = fmin * 2.0 ** (np.r_[0:n_bins] / np.float(bins_per_octave))
    else:
        warnings.warn("If nmax is given, n_bins will be ignored", SyntaxWarning)
        n_bins = np.ceil(bins_per_octave * np.log2(fmax / fmin))
        freqs = fmin * 2.0 ** (np.r_[0:n_bins] / np.float(bins_per_octave))
    if np.max(freqs) > fs / 2 and topbin_check:
        raise ValueError(f"The top bin {np.max(freqs)} Hz has exceeded the Nyquist frequency, \
                           please reduce the `n_bins`")
    
    kernel = np.zeros((int(n_bins), int(fft_len)), dtype=np.complex64)
    
    length = np.ceil(q * fs / freqs)
    for k in range(0, int(n_bins)):
        freq = freqs[k]
        l = np.ceil(q * fs / freq)
        
        if l % 2 == 1:
            start = int(np.ceil(fft_len / 2.0 - l / 2.0)) - 1
        else:
            start = int(np.ceil(fft_len / 2.0 - l / 2.0))

        sig = get_window(window, int(l), fftbins=True) * np.exp(
            np.r_[-l // 2:l // 2] * 1j * 2 * np.pi * freq / fs) / l
        
        if norm:
            kernel[k, start:start + int(l)] = sig / np.linalg.norm(sig, norm)
        else:
            kernel[k, start:start + int(l)] = sig
    return kernel, fft_len, length, freqs

def _nextpow2(a: float) -> int:
    return int(np.ceil(np.log2(a)))

# Function to prepare cqt kernel
def prepare_cqt_kernel(
    sr=22050,
    hop_length=512,
    fmin=32.70,
    fmax=None,
    n_bins=84,
    bins_per_octave=12,
    norm=1,
    filter_scale=1,
    window="hann"
):
    q = float(filter_scale) / (2 ** (1 / bins_per_octave) - 1)
    print(q)
    return create_cqt_kernels(q, sr, fmin, n_bins, bins_per_octave, norm, window, fmax)
# Function to create cqt image
def create_cqt_image(wave, hop_length=16):
    CQTs = []
    for i in range(3):
        x = wave[i]
        x = tf.expand_dims(tf.expand_dims(x, 0), 2)
        x = tf.pad(x, PADDING, "REFLECT")

        CQT_real = tf.nn.conv1d(x, CQT_KERNELS_REAL, stride=hop_length, padding="VALID")
        CQT_imag = -tf.nn.conv1d(x, CQT_KERNELS_IMAG, stride=hop_length, padding="VALID")
        CQT_real *= tf.math.sqrt(LENGTHS)
        CQT_imag *= tf.math.sqrt(LENGTHS)

        CQT = tf.math.sqrt(tf.pow(CQT_real, 2) + tf.pow(CQT_imag, 2))
        CQTs.append(CQT[0])
    return tf.stack(CQTs, axis=2)

HOP_LENGTH = 64
cqt_kernels, KERNEL_WIDTH, lengths, _ = prepare_cqt_kernel(
    sr=2048,
    hop_length=HOP_LENGTH,
    fmin=20,
    fmax=500,
    bins_per_octave=12)
LENGTHS = tf.constant(lengths, dtype=tf.float32)
CQT_KERNELS_REAL = tf.constant(np.swapaxes(cqt_kernels.real[:, np.newaxis, :], 0, 2))
CQT_KERNELS_IMAG = tf.constant(np.swapaxes(cqt_kernels.imag[:, np.newaxis, :], 0, 2))
PADDING = tf.constant([[0, 0],
                        [KERNEL_WIDTH // 2, KERNEL_WIDTH // 2],
                        [0, 0]])

<a id="3"></a>
<h1 style='background:#1EA1A1; border:0; color:white'><center> Reading TFRecords<center></h1>

In [ ]:
def read_labeled_tfrecord(example):
    tfrec_format = {
        "wave": tf.io.FixedLenFeature([], tf.string),
        "wave_id": tf.io.FixedLenFeature([], tf.string),
        "target": tf.io.FixedLenFeature([], tf.int64)
    }
    example = tf.io.parse_single_example(example, tfrec_format)
    return prepare_image(example["wave"], config.image_size), tf.reshape(tf.cast(example["target"], tf.float32), [1])


def read_unlabeled_tfrecord(example, return_image_id):
    tfrec_format = {
        "wave": tf.io.FixedLenFeature([], tf.string),
        "wave_id": tf.io.FixedLenFeature([], tf.string)
    }
    example = tf.io.parse_single_example(example, tfrec_format)
    return prepare_image(example["wave"], config.image_size), example["wave_id"] if return_image_id else 0

def count_data_items(fileids):
    return len(fileids) * 28000

def count_data_items_test(fileids):
    return len(fileids) * 22600

def prepare_image(wave, dim= config.image_size):
    wave = tf.reshape(tf.io.decode_raw(wave, tf.float64), (3, 4096))
    normalized_waves = []
    for i in range(3):
        normalized_wave = wave[i] / tf.math.reduce_max(wave[i])
        normalized_waves.append(normalized_wave)
    wave = tf.stack(normalized_waves)
    wave = tf.cast(wave, tf.float32)
    image = create_cqt_image(wave, HOP_LENGTH)
    image = tf.image.resize(image, size=(dim, dim))
    return tf.reshape(image, (dim, dim, 3))

def mixup(image, label, PROBABILITY = 1.0, AUG_BATCH= config.batch_size):
    # input image - is a batch of images of size [n,dim,dim,3] not a single image of [dim,dim,3]
    # output - a batch of images with mixup applied
    DIM = config.image_size
    
    imgs = []; labs = []
    for j in range(AUG_BATCH):
        # DO MIXUP WITH PROBABILITY DEFINED ABOVE
        P = tf.cast( tf.random.uniform([],0,1)<=PROBABILITY, tf.float32)
        # CHOOSE RANDOM
        k = tf.cast( tf.random.uniform([],0,AUG_BATCH),tf.int32)
        a = tf.random.uniform([],0,1)*P # this is beta dist with alpha=1.0
        # MAKE MIXUP IMAGE
        img1 = image[j,]
        img2 = image[k,]
        imgs.append((1-a)*img1 + a*img2)
        # MAKE CUTMIX LABEL
        lab1 = label[j,]
        lab2 = label[k,]
        labs.append((1-a)*lab1 + a*lab2)
            
    # RESHAPE HACK SO TPU COMPILER KNOWS SHAPE OF OUTPUT TENSOR (maybe use Python typing instead?)
    image2 = tf.reshape(tf.stack(imgs),(AUG_BATCH,DIM,DIM,3))
    label2 = tf.reshape(tf.stack(labs),(AUG_BATCH,))
    return image2,label2

def time_shift(img, shift= config.t_shift):
    if shift > 0:
        T = config.image_size
        P = tf.random.uniform([],0,1)
        SHIFT = tf.cast(T * P, tf.int32)
        return tf.concat([img[-SHIFT:], img[:-SHIFT]], axis=0)
    return img


def spector_shift(img, shift= config.s_shift):
    if shift > 0:
        T = config.image_size
        P = tf.random.uniform([],0,1)
        SHIFT = tf.cast(T * P, tf.int32)
        return tf.concat([img[:, -SHIFT:], img[:, :-SHIFT]], axis=1)
    return img
  
def rotate(img, angle= config.r_angle):
    if angle > 0:
        P = tf.random.uniform([],0,1)
        A = tf.cast(angle * P, tf.float32)
        return tfa.image.rotate(img, A)
    return img

def img_aug_f(img):
    img = time_shift(img)
    img = spector_shift(img)
    # img = rotate(img)
    return img

def imgs_aug_f(imgs, batch_size):
    _imgs = []
    DIM = config.image_size
    for j in range(batch_size):
        _imgs.append(img_aug_f(imgs[j]))
    return tf.reshape(tf.stack(_imgs),(batch_size,DIM,DIM,3))

def label_positive_shift(labels):
    return labels *  config.label_positive_shift

def aug_f(imgs, labels, batch_size):
    imgs, label = mixup(imgs, labels, config.mix_up_p, batch_size)
    imgs = imgs_aug_f(imgs, batch_size)
    return imgs, label_positive_shift(label)


<a id="4"></a>
<h1 style='background:#1EA1A1; border:0; color:white'><center> 📊 Dataset Creation <center></h1>

In [ ]:
def get_dataset(files, shuffle = False, repeat = False, labeled=True, return_image_ids=True, batch_size=config.batch_size ,aug=False):
    
    ds = tf.data.TFRecordDataset(files, num_parallel_reads=config.AUTO, compression_type="GZIP")
    ds = ds.cache()
    
    if repeat:
        ds = ds.repeat()
    
    if shuffle: 
        ds = ds.shuffle(2048)
        opt = tf.data.Options()
        opt.experimental_deterministic = False
        ds = ds.with_options(opt)
        
    if labeled: 
        ds = ds.map(read_labeled_tfrecord, num_parallel_calls=config.AUTO)
    else:
        ds = ds.map(read_unlabeled_tfrecord,num_parallel_calls= config.AUTO) 
    
    ds = ds.batch(batch_size)
    if aug:
        ds = ds.map(lambda x, y: aug_f(x, y, batch_size), num_parallel_calls=config.AUTO)
    
    ds = ds.prefetch(config.AUTO)
    return ds


<a id="5"></a>
<h1 style='background:#1EA1A1; border:0; color:white'><center> 🧠 Modelling <center></h1>


In [ ]:
def create_model(backbone_model):
    x = GlobalAveragePooling2D()(backbone_model.output)
    prediction_layer = Dense(1, activation = 'sigmoid')(x)
    
    model = Model(inputs = backbone_model.input , outputs = prediction_layer)
    
    return model

<a id="6"></a>
<h1 style='background:#1EA1A1; border:0; color:white'><center> 🔄 KFold Training <center></h1>

In [ ]:
def get_lr_callback():
    lr_start   = 1e-4
    lr_max     = 0.000015 * config.batch_size 
    lr_min     = 1e-7
    lr_ramp_ep = 3
    lr_sus_ep  = 0
    lr_decay   = 0.7
   
    def lrfn(epoch):
        if epoch < lr_ramp_ep:
            lr = (lr_max - lr_start) / lr_ramp_ep * epoch + lr_start
            
        elif epoch < lr_ramp_ep + lr_sus_ep:
            lr = lr_max
            
        else:
            lr = (lr_max - lr_min) * lr_decay**(epoch - lr_ramp_ep - lr_sus_ep) + lr_min
            
        return lr

    lr_callback = tf.keras.callbacks.LearningRateScheduler(lrfn, verbose=True)
    return lr_callback

In [ ]:
def vis_lr_callback():
    lr_start   = 1e-4
    lr_max     = 0.000015 * config.batch_size
    lr_min     = 1e-7
    lr_ramp_ep = 3
    lr_sus_ep  = 0
    lr_decay   = 0.7
   
    def lrfn(epoch):
        if epoch < lr_ramp_ep:
            lr = (lr_max - lr_start) / lr_ramp_ep * epoch + lr_start
            
        elif epoch < lr_ramp_ep + lr_sus_ep:
            lr = lr_max
            
        else:
            lr = (lr_max - lr_min) * lr_decay**(epoch - lr_ramp_ep - lr_sus_ep) + lr_min
            
        return lr

    plt.figure(figsize=(10, 7))
    plt.plot([lrfn(i) for i in range(config.epochs)])
    plt.show()
vis_lr_callback()

In [ ]:
histories = []
skf = KFold(n_splits= 4 , shuffle = True , random_state = config.seed)
train_files = np.array(train_files)
for k,(train_idx,valid_idx) in enumerate(skf.split(train_files)):
    print(f"***************** RUNNING FOLD {k+1} *****************")
    train_file_list = train_files[train_idx]
    valid_file_list = train_files[valid_idx]
    print(f"number of train files {len(train_file_list)} number of valid files {len(valid_file_list)}")
    
    #preparing dataset
    print("preparing train datasets")
    train_dataset = get_dataset(train_file_list, batch_size= config.batch_size, shuffle=True, repeat=True, aug=True)
    print("preparing validation datasets")
    valid_dataset = get_dataset(valid_file_list, batch_size= config.batch_size, repeat=False, shuffle=False, aug=False)
     
    train_steps = count_data_items(train_file_list)/config.batch_size//4
    validation_steps = count_data_items(valid_file_list)/config.batch_size//4
    print(f'Training Steps {train_steps} , validation steps {validation_steps}')

    #clearing session
    print("clearning session")
    K.clear_session()
    
    with strategy.scope():
        backbone_model = efn.EfficientNetB1(include_top=False,weights='imagenet',input_shape=(config.image_size,config.image_size,3))
        model = create_model(backbone_model)
        lr_decayed_fn = tf.keras.experimental.CosineDecay(1e-3, train_steps)
        opt = tfa.optimizers.AdamW(lr_decayed_fn, learning_rate=1e-4)
        model.compile( 
            optimizer = opt,
            loss = BinaryCrossentropy(),
            metrics = 'AUC'
        )
    model_checkpoint = ModelCheckpoint(f'{config.dir_path}/eff_net_fold_{k+1}.h5',
                                       monitor = 'val_auc', verbose = 1, 
                                       save_best_only = True, save_weights_only = True, mode = 'max') 
    hist = model.fit(train_dataset,steps_per_epoch=train_steps, validation_data= valid_dataset,
                      epochs = config.epochs , callbacks =[model_checkpoint , get_lr_callback()])
    histories.append(hist)

<a id = "7"> </a>
<h1 style='background:#1EA1A1; border:0; color:white'><center> 📌 References <center></h1>
 
 **Thank You Hidehisa Arai for providing such good kernels**
 * Hidehisa's [Training Notebook](https://www.kaggle.com/hidehisaarai1213/g2net-tf-on-the-fly-cqt-tpu-training) and [Inference Notebook](https://www.kaggle.com/hidehisaarai1213/g2net-tf-on-the-fly-cqt-tpu-inference)
 * Hidehisa's [Train Dataset](https://www.kaggle.com/hidehisaarai1213/g2net-waveform-tfrecords-train-0-4) and [Test Dataset](https://www.kaggle.com/hidehisaarai1213/g2net-waveform-tfrecords-test-0-4)

**Thanks for viewing, drop your suggestions down in the comments below. 🙂**
